<a href="https://colab.research.google.com/github/srujaan/DL-with-PyTorch/blob/master/find_units.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch

In [14]:
# "t_c" is temperature in celcius
# "t_u" is temperature are unknown units
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [15]:
# model
def model(t_u, w, b):
  return w * t_u + b

In [16]:
# loss function
def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c) ** 2
  return squared_diffs.mean()

In [17]:
# initialize w,b parameteres
w = torch.ones(())
b = torch.ones(())

In [18]:
t_p = model(t_u, w, b)
t_p

tensor([36.7000, 56.9000, 59.2000, 82.9000, 57.3000, 49.9000, 34.9000, 22.8000,
        49.4000, 61.4000, 69.4000])

In [19]:
# check value of loss
loss = loss_fn(t_p, t_c)
loss

tensor(1847.4844)

In [20]:
# Decreasing loss

delta = 0.1

loss_rate_of_change_w = \
  (loss_fn(model(t_u, w + delta, b), t_c) - 
   loss_fn(model(t_u, w - delta, b), t_c)) / (2.0 * delta)

In [21]:
# learning rate should be low as possible

learning_rate = 1e-2
w = w - learning_rate * loss_rate_of_change_w

In [22]:
# we can do same for b

loss_rate_of_change_b = \
  (loss_fn(model(t_u, w, b + delta), t_c) - 
   loss_fn(model(t_u, w, b - delta), t_c)) / (2.0 * delta)

b = b - learning_rate * loss_rate_of_change_b

In [23]:
def dloss_fn(t_p, t_c):
  dsq_diffs = 2 * (t_p - t_c) / t_p.size(0) # The division is from the derivative mean
  return dsq_diffs


In [24]:
# Applying derivatives to the model

def dmodel_dw(t_u, w, b):
  return t_u

def dmodel_db(t_u, w, b):
  return 1.0

In [26]:
# Putting all this together

def grad_fn(t_u, t_c, t_p, w, b):
  dloss_dtp = dloss_fn(t_p, t_c)
  dloss_dw = dloss_dtp * dmodel_dw(t_u, w, b)
  dloss_db = dloss_dtp * dmodel_db(t_u, w, b)
  return torch.stack([dloss_dw.sum(), dloss_db.sum()])

In [27]:
# Training loop

def training_loop(n_epochs, learning_rate, params, t_u, t_c):
  for epoch in range(1, n_epochs + 1):
    w, b = params

    t_p = model(t_u, w, b)  # Forward pass
    loss = loss_fn(t_p, t_c)
    grad = grad_fn(t_u, t_c, t_p, w, b) # Backward pass

    params = params - learning_rate * grad

    print('Epoch %d, Loss %f' % (epoch, float(loss)))

  return params
